In [1]:
import io
import zipfile
import requests
import frontmatter

/Users/shrutroup/git_repos/ai_hero/course/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [25]:
import io
import zipfile
import requests
import frontmatter

def read_repo_data(repo_owner, repo_name):
    """
    Download and parse all markdown files from a GitHub repository.
    
    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name
    
    Returns:
        List of dictionaries containing file content and metadata
    """
    prefix = 'https://codeload.github.com' 
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main'
    resp = requests.get(url)
    
    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))
    
    for file_info in zf.infolist():
        filename = file_info.filename
        filename_lower = filename.lower()

        if not (filename_lower.endswith('.md') 
            or filename_lower.endswith('.mdx')):
            continue
    
        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                post = frontmatter.loads(content)
                data = post.to_dict()
                data['filename'] = filename
                repository_data.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue
    
    zf.close()
    return repository_data



In [29]:
#dtc_faq = read_repo_data('DataTalksClub', 'faq')
evidently_docs = read_repo_data('evidentlyai', 'docs')

#print(f"FAQ documents: {len(dtc_faq)}")
print(f"Evidently documents: {len(evidently_docs)}")


Evidently documents: 95


In [30]:
len(evidently_docs[45]['content'])

21712

## Simple Splitting - by length

In [58]:
def sliding_window(seq, size, step):
    if size <=0 or step <=0:
        raise ValueError("size and step must be positive")
    n = len(seq)
    result = []
    for i in range(0, n, step):
        chunk = seq[i:i+size]
        result.append({'start': i, 'chunk': chunk})
        if i + size >= n:
            break
    return result

#doc_content = evidently_docs[45].pop('content')
chunked_seq = sliding_window(seq=evidently_docs[45]['content'], size=2000, step=1000)

In [59]:
chunked_seq

[{'start': 0,
  'chunk': "In this tutorial, you will learn how to perform regression testing for LLM outputs.\n\nYou can compare new and old responses after changing a prompt, model, or anything else in your system. By re-running the same inputs with new parameters, you can spot any significant changes. This helps you push updates with confidence or identify issues to fix.\n\n<Info>\n  **This example uses Evidently Cloud.** You'll run evals in Python and upload them. You can also skip the upload and view Reports locally. For self-hosted, replace `CloudWorkspace` with `Workspace`.\n</Info>\n\n# Tutorial scope\n\nHere's what we'll do:\n\n* **Create a toy dataset**. Build a small Q&A dataset with answers and reference responses.\n\n* **Get new answers**. Imitate generating new answers to the same question.\n\n* **Create and run a Report with Tests**. Compare the answers using LLM-as-a-judge to evaluate length, correctness and style consistency.\n\n* **Build a monitoring Dashboard**. Get p

In [63]:
all_chunked_docs = []
for i in range(len(evidently_docs)):
    chunked_seq = sliding_window(seq=evidently_docs[i]['content'], size=2000, step=1000)
    
    all_chunked_docs.extend(chunked_seq)

all_chunked_docs[0:10]

[{'start': 0,
  'chunk': '<Note>\n  If you\'re not looking to build API reference documentation, you can delete\n  this section by removing the api-reference folder.\n</Note>\n\n## Welcome\n\nThere are two ways to build API documentation: [OpenAPI](https://mintlify.com/docs/api-playground/openapi/setup) and [MDX components](https://mintlify.com/docs/api-playground/mdx/configuration). For the starter kit, we are using the following OpenAPI specification.\n\n<Card\n  title="Plant Store Endpoints"\n  icon="leaf"\n  href="https://github.com/mintlify/starter/blob/main/api-reference/openapi.json"\n>\n  View the OpenAPI specification file\n</Card>\n\n## Authentication\n\nAll API endpoints are authenticated using Bearer tokens and picked up from the specification file.\n\n```json\n"security": [\n  {\n    "bearerAuth": []\n  }\n]\n```'},
 {'start': 0,
  'chunk': '<Update label="2025-07-18" description="Evidently v0.7.11">\n  ## **Evidently 0.7.11**\n\n  Full release notes on [Github](https://gi

In [61]:
len(all_chunked_docs)

575

## Splitting my Paragraphs and Sections
Using newline to split

In [62]:
import re
text = evidently_docs[45]['content']
paragraphs = re.split(r"\n\s*\n", text.strip())
paragraphs

['In this tutorial, you will learn how to perform regression testing for LLM outputs.',
 'You can compare new and old responses after changing a prompt, model, or anything else in your system. By re-running the same inputs with new parameters, you can spot any significant changes. This helps you push updates with confidence or identify issues to fix.',
 "<Info>\n  **This example uses Evidently Cloud.** You'll run evals in Python and upload them. You can also skip the upload and view Reports locally. For self-hosted, replace `CloudWorkspace` with `Workspace`.\n</Info>",
 '# Tutorial scope',
 "Here's what we'll do:",
 '* **Create a toy dataset**. Build a small Q&A dataset with answers and reference responses.',
 '* **Get new answers**. Imitate generating new answers to the same question.',
 '* **Create and run a Report with Tests**. Compare the answers using LLM-as-a-judge to evaluate length, correctness and style consistency.',
 '* **Build a monitoring Dashboard**. Get plots to track th

## Splitting by Sections
Taking advantage of the documents structure. Markdown documents have this structure

In [80]:
import re
level=2
text = '## 1. Installation and Imports\n\nInstall Evidently:\n\n```python\npip install evidently[llm] \n```\n\nImport the required modules:\n\n```python\nimport pandas as pd\nfrom evidently.future.datasets import Dataset\nfrom evidently.future.datasets import DataDefinition\nfrom evidently.future.datasets import Descriptor\nfrom evidently.future.descriptors import *\nfrom evidently.future.report import Report\nfrom evidently.future.presets import TextEvals\nfrom evidently.future.metrics import *\nfrom evidently.future.tests import *\n\nfrom evidently.features.llm_judge import BinaryClassificationPromptTemplate\n```\n\nTo connect to Evidently Cloud:\n\n```python\nfrom evidently.ui.workspace.cloud import CloudWorkspace\n```\n\n**Optional.** To create monitoring panels as code:\n\n```python\nfrom evidently.ui.dashboards import DashboardPanelPlot\nfrom evidently.ui.dashboards import DashboardPanelTestSuite\nfrom evidently.ui.dashboards import DashboardPanelTestSuiteCounter\nfrom evidently.ui.dashboards import TestSuitePanelType\nfrom evidently.ui.dashboards import ReportFilter\nfrom evidently.ui.dashboards import PanelValue\nfrom evidently.ui.dashboards import PlotType\nfrom evidently.ui.dashboards import CounterAgg\nfrom evidently.tests.base_test import TestStatus\nfrom evidently.renderers.html_widgets import WidgetSize\n```\n\nPass your OpenAI key:\n\n```python\nimport os\nos.environ["OPENAI_API_KEY"] = "YOUR KEY"\n```'
header_pattern = r'^(#{' + str(level) + r'} )(.+)$'
pattern = re.compile(header_pattern, re.MULTILINE)
pattern

re.compile(r'^(#{2} )(.+)$', re.MULTILINE|re.UNICODE)

In [81]:
match = re.search(pattern, text)
print(match.group(0))
print(match.group(1))
print(match.group(2))

## 1. Installation and Imports
## 
1. Installation and Imports


In [82]:
# Split and keep the headers
parts = pattern.split(text)
parts

['',
 '## ',
 '1. Installation and Imports',
 '\n\nInstall Evidently:\n\n```python\npip install evidently[llm] \n```\n\nImport the required modules:\n\n```python\nimport pandas as pd\nfrom evidently.future.datasets import Dataset\nfrom evidently.future.datasets import DataDefinition\nfrom evidently.future.datasets import Descriptor\nfrom evidently.future.descriptors import *\nfrom evidently.future.report import Report\nfrom evidently.future.presets import TextEvals\nfrom evidently.future.metrics import *\nfrom evidently.future.tests import *\n\nfrom evidently.features.llm_judge import BinaryClassificationPromptTemplate\n```\n\nTo connect to Evidently Cloud:\n\n```python\nfrom evidently.ui.workspace.cloud import CloudWorkspace\n```\n\n**Optional.** To create monitoring panels as code:\n\n```python\nfrom evidently.ui.dashboards import DashboardPanelPlot\nfrom evidently.ui.dashboards import DashboardPanelTestSuite\nfrom evidently.ui.dashboards import DashboardPanelTestSuiteCounter\nfrom e

In [104]:
def split_markdown_by_level(text, level=2):

    """

    Split markdown text by a specific header level.
    :param text: Markdown text as a string
    :param level: Header level to split on
    :return: List of sections as strings
    """

    # This regex matches markdown headers
    # For level 2, it matches lines starting with "## "

    header_pattern = r'^(#{' + str(level) + r'} )(.+)$'
    pattern = re.compile(header_pattern, re.MULTILINE)
    # Split and keep the headers
    parts = pattern.split(text)
    sections = []
    for i in range(1, len(parts), 3):

        # We step by 3 because regex.split() with
        # capturing groups returns:
        # [before_match, group1, group2, after_match, ...]
        # here group1 is "## ", group2 is the header text

        header = parts[i] + parts[i+1]  # "## " + "Title"
        header = header.strip()
        
        # Get the content after this header
        content = ""
        if i+2 < len(parts):
            content = parts[i+2].strip()
        if content:
            section = f'{header}\n\n{content}'
        else:
            section = header
        sections.append(section)

    return sections

In [105]:
sections = split_markdown_by_level(evidently_docs[45]['content'], level=2)
sections

['## 1. Installation and Imports\n\nInstall Evidently:\n\n```python\npip install evidently[llm] \n```\n\nImport the required modules:\n\n```python\nimport pandas as pd\nfrom evidently.future.datasets import Dataset\nfrom evidently.future.datasets import DataDefinition\nfrom evidently.future.datasets import Descriptor\nfrom evidently.future.descriptors import *\nfrom evidently.future.report import Report\nfrom evidently.future.presets import TextEvals\nfrom evidently.future.metrics import *\nfrom evidently.future.tests import *\n\nfrom evidently.features.llm_judge import BinaryClassificationPromptTemplate\n```\n\nTo connect to Evidently Cloud:\n\n```python\nfrom evidently.ui.workspace.cloud import CloudWorkspace\n```\n\n**Optional.** To create monitoring panels as code:\n\n```python\nfrom evidently.ui.dashboards import DashboardPanelPlot\nfrom evidently.ui.dashboards import DashboardPanelTestSuite\nfrom evidently.ui.dashboards import DashboardPanelTestSuiteCounter\nfrom evidently.ui.das

In [106]:
# process all the whole evidently doc
evidently_chunks = []


for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    sections = split_markdown_by_level(doc_content, level=2)
    for section in sections:
        section_doc = doc_copy.copy()
        section_doc['section'] = section
        evidently_chunks.append(section_doc)

In [117]:
evidently_chunks[1]

{'title': 'Introduction',
 'description': 'Example section for showcasing API endpoints',
 'filename': 'docs-main/api-reference/introduction.mdx',
 'section': '## Authentication\n\nAll API endpoints are authenticated using Bearer tokens and picked up from the specification file.\n\n```json\n"security": [\n  {\n    "bearerAuth": []\n  }\n]\n```'}

In [118]:
evidently_docs[3]

{'title': 'Introduction',
 'description': 'Example section for showcasing API endpoints',
 'content': '<Note>\n  If you\'re not looking to build API reference documentation, you can delete\n  this section by removing the api-reference folder.\n</Note>\n\n## Welcome\n\nThere are two ways to build API documentation: [OpenAPI](https://mintlify.com/docs/api-playground/openapi/setup) and [MDX components](https://mintlify.com/docs/api-playground/mdx/configuration). For the starter kit, we are using the following OpenAPI specification.\n\n<Card\n  title="Plant Store Endpoints"\n  icon="leaf"\n  href="https://github.com/mintlify/starter/blob/main/api-reference/openapi.json"\n>\n  View the OpenAPI specification file\n</Card>\n\n## Authentication\n\nAll API endpoints are authenticated using Bearer tokens and picked up from the specification file.\n\n```json\n"security": [\n  {\n    "bearerAuth": []\n  }\n]\n```',
 'filename': 'docs-main/api-reference/introduction.mdx'}

## Intelligent Chunking with LLM

In some cases, we want to be more intelligent with chunking. Instead of doing simple splits, we delegate this work to AI.
This makes sense when:

Complex structure: Documents have complex, non-standard structure
Semantic coherence: You want chunks that are semantically meaningful
Custom logic: You need domain-specific splitting rules
Quality over cost: You prioritize quality over processing cost

This costs money. In most cases, we don't need intelligent chunking.
Simple approaches are sufficient. Use intelligent chunking only when


You already evaluated simpler methods and you can confirm that they produce poor results
You have complex, unstructured documents
Quality is more important than cost
You have the budget for LLM processing
